PyTorch 의 공식 튜토리얼에는 LSTM-CRF 를 구현하는 예시가 있습니다. 아래의 튜토리얼에 뉴스 데이터를 이용하여 character-level NER tagger 를 만들어 봅니다.

https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

In [1]:
import numpy as np
import pickle
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

torch.manual_seed(1)

튜토리얼의 `prepare_sequence` 에는 `unknown_idx` 가 없습니다. 우리가 알지 못하는 글자가 입력될 수 있으니 이 기능을 추가합니다.

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix, unknown_idx):    
    idxs = [to_ix.get(w, unknown_idx) for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

뉴스를 list 형식으로 모두 불러옵니다.

In [3]:
import config
from navernews_10days import get_news_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(date='2016-10-20')
sents = list(DoublespaceLineCorpus(path, iter_sent=True))

soynlp=0.0.492
added lovit_textmining_dataset


RNN 은 batch 방식으로 구현하지 않으면 학습이 느린 편입니다. 하지만 이 부분은 일단 RNN 모델이 익숙해진 다음에 연습해도 됩니다.

일단은 아래의 세 단어가 어절에서 명사로 입력된 문장 752 개만 toydata 로 선택합니다.

In [4]:
seed_words = set(
    '''몬스타엑스 아이오아이 트와이스'''.split())

def contain_seed(sent):
    for seed in seed_words:
        if seed in sent:
            return True
    return False

tokenizer = LTokenizer(scores={seed:1 for seed in seed_words})
train_sents = [tokenizer.tokenize(sent) for sent in sents if contain_seed(sent)]
len(train_sents)

752

이 문장에 등장한 글자를 index 로 변환하는 함수를 만듭니다. 등장 빈도가 2 이하인 글자는 모두 unk 처리합니다.

In [5]:
from collections import Counter

def scan_character(sents, min_count=3):
    char_counter = Counter(c for sent in sents for word in sent for c in word)
    char_counter = {c:count for c, count in char_counter.items() if count >= min_count}
    idx_to_char = [c for c in sorted(char_counter, key=lambda x:-char_counter[x])]
    char_to_idx = {char:idx for idx, char in enumerate(idx_to_char)}
    return idx_to_char, char_to_idx

idx_to_char, char_to_idx = scan_character(train_sents)
idx_to_char.append('#Unk')
unknown_idx = len(char_to_idx)
unknown_idx

633

In [6]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {"O": 0, "B": 1, "I": 2, START_TAG: 3, STOP_TAG: 4}

In [7]:
class CharNERDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

앞선 예시처럼 데이터 생성 함수를 만듭니다. seed words 의 시작 부분을 1 로, 나머지 부분을 2로 만들면 됩니다.

In [8]:
def create_dataset(char_to_idx, sents, seed_words, unknown_idx, test_data=False):

    X = []
    Y = []
    for words in sents:
        chars = ''.join(words)
        n_chars = len(chars)
        y = np.zeros(n_chars, dtype=np.int)

        b = 0
        for word in words:
            e = b + len(word)
            if word in seed_words:
                y[b:e] = 2
                y[b] = 1
            b = e

        if (not test_data) and (y.sum() == 0):
            continue

        x = prepare_sequence(chars, char_to_idx, unknown_idx)
        X.append(torch.LongTensor(x))
        Y.append(torch.LongTensor(y))

    dataset = CharNERDataset(X, Y)
    return dataset

dataset = create_dataset(char_to_idx, train_sents, seed_words, unknown_idx)

In [9]:
len(dataset)

752

데이터와 글자열의 예시입니다.

In [10]:
print(dataset[1])
print(''.join(idx_to_char[idx] for idx in dataset[1][0]))

(tensor([428,  53, 206,  39,   0, 174,  42,  81,  12,  46, 174,  29,  42,  22,
        351, 244, 298, 101, 211,   3,   9,   0,   2, 110,  27,  12,  11, 559,
          2,  25, 322, 158]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0,
        0, 0, 0, 0, 0, 0, 0, 0]))
머니투데이박진영기자박해진지창욱황치열트와이스등인기한류스타참여


In [11]:
print(dataset[10])
print(''.join(idx_to_char[idx] for idx in dataset[10][0]))

(tensor([200, 168,  92,   8,   0,  13,   8,   0,   4,  99, 352, 182, 151,  14,
        300,   2,   5,  15,   5,   7, 126,  56,  35, 283,   0, 191,   6, 633,
         19,  49,  41,  45, 467,  11,  38,  12, 116,  98,   0,   0, 510,  48,
          0,   1,   0,   7,  30,  29, 145,  97, 228,  23,  23,  13, 269,   8,
        440,   7,  77,  91,  62,  95,  47, 254,  21,  77,  56,   0,   9,  56,
        441,   6,  35,   1,  13, 269, 121, 137,  60,   3,   1,  60,   7,  33,
         44,  80, 111,  62,   0,  40,  52,  22, 353,  23,   8,   0,  13,   8,
          0, 105, 208, 468,  26,   0, 194,  34,  53,  27,  37,  20,  21, 219,
         16, 255,   1,  19, 149,  35,   1]), tensor([0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2,
        2,

아래의 코드는 공식 튜토리얼에서 그대로 가져왔습니다. LSTM 의 prediction 뿐 아니라 beam search, loss 부분이 모두 구현되어 있습니다. 아래처럼 예측된 태그의 점수와 실제 학습데이터의 점수의 차를 log likelihood 로 정의합니다.

```python
    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
```

In [12]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

앞서 만들어둔 모델이 있어서 이를 로딩합니다.

In [13]:
model_path = 'lstmcrf_ner_sample.pt'
model = torch.load(model_path)

In [69]:
EMBEDDING_DIM = 16
HIDDEN_DIM = 64
epochs = 10


model = BiLSTM_CRF(len(char_to_idx)+1, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

n_data = len(dataset)

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(epochs):
    loss_sum = 0
    for i, (x, y) in enumerate(dataset):
        model.zero_grad()
        loss = model.neg_log_likelihood(x, y)
        loss.backward()        
        optimizer.step()
        loss_sum += loss.data.numpy()
        if i % 100 == 0:
            print('\repoch = {} / {}, sent = {} / {}'.format(epoch+1, epochs, i, n_data), end='')
    loss_avg = float(loss_sum / len(dataset))
    print('\repoch = {} / {}, loss = {:.6}{}'.format(epoch+1, epochs, loss_avg, ' '*20))

epoch = 1 / 10, loss = 2.85021                    
epoch = 2 / 10, loss = 0.301758                    
epoch = 3 / 10, loss = 0.190214                    
epoch = 4 / 10, loss = 0.13218                    
epoch = 5 / 10, loss = 0.100868                    
epoch = 6 / 10, loss = 0.091456                    
epoch = 7 / 10, loss = 0.0816277                    
epoch = 8 / 10, loss = 0.0648198                    
epoch = 9 / 10, loss = 0.0628505                    
epoch = 10 / 10, loss = 0.0562666                    


문장에서 단어열과 띄어쓰기 태그열을 분해하는 함수입니다.

In [14]:
def sent_to_chartags(sent, nonspace=0, space=1):
    chars = sent.replace(' ','')
    if not chars:
        return '', []

    tags = [nonspace]*(len(chars) - 1) + [space]
    idx = 0
    for c in sent:
        if c == ' ':
            tags[idx-1] = space
        else:
            idx += 1
    return chars, tags

sent_to_chartags('예문 입니다')

('예문입니다', [0, 1, 0, 0, 1])

문장이 입력되면 글자열을 먼저 분해한 뒤, 이를 모델에 입력할 수 있도록 글자 index 로 변환합니다. tagging 을 한 뒤 BIO tag 를 이용하여 객체명 주변에 대괄호를 칩니다.

In [15]:
def tag(sent, char_to_idx, unknown_idx, model):
    chars, spaces = sent_to_chartags(sent)
    x = prepare_sequence(chars, char_to_idx, unknown_idx)
    score, tags = model(x)

    sent_ = []
    n = len(chars)
    for i, (char, space, tag) in enumerate(zip(chars, spaces, tags)):
        if tag == 1:
            sent_.append('[')
        sent_.append(char)
        if (tag == 2) and (i == n-1 or tags[i+1] == 0):
            sent_.append(']')
        if space == 1:
            sent_.append(' ')
    return ''.join(sent_)

sent = '음악방송에 트와이스가 출연하였습니다'
tag(sent, char_to_idx, unknown_idx, model)

'음악방송에 [트와이스]가 출연하였습니다 '

In [16]:
sent = '예능프로그램에 트와이스의 맴버가 출연했습니다'
tag(sent, char_to_idx, unknown_idx, model)

'예능프로그램에 [트와이스]의 맴버가 출연했습니다 '

In [17]:
sent = '24일 컴백을 앞둔 트아이스는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다'
tag(sent, char_to_idx, unknown_idx, model)

'24일 컴백을 앞둔 [트아이스]는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다 '

In [18]:
sent = '24일 컴백을 앞둔 트하이스는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다'
tag(sent, char_to_idx, unknown_idx, model)

'24일 컴백을 앞둔 트[하이스]는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다 '

In [19]:
sent = '24일 컴백을 앞둔 트아히스는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다'
tag(sent, char_to_idx, unknown_idx, model)

'24일 컴백을 앞둔 [트아히스]는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다 '

In [20]:
sent = '24일 컴백을 앞둔 씨스타는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다'
tag(sent, char_to_idx, unknown_idx, model)

'24일 컴백을 앞둔 씨스타는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다 '

In [21]:
sent = '24일 컴백을 앞둔 몬스타엑스는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다'
tag(sent, char_to_idx, unknown_idx, model)

'24일 컴백을 앞둔 [몬스타엑스]는 두번째 뮤직비디오 티저 영상 앨범 하이라이트 등을 공개하며 컴백 프로모션을 이어갈 계획이다 '

In [25]:
# model_path = 'lstmcrf_ner_sample.pt'
# torch.save(model, model_path)